In [5]:
from langchain_community.chat_models import ChatOpenAI
import dotenv
import os
import joblib
from discussion_agents.cog.agent.expel import ExpeLAgent
from discussion_agents.cog.modules.memory.expel import ExpeLExperienceMemory

dotenv.load_dotenv("../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", openai_api_key=openai_api_key)
hotpotqa = joblib.load('../../../tests/assets/hotpotqa/hotpot-qa-distractor-sample.joblib')
experiences = joblib.load('../../../tests/assets/expel/expel_experiences_10_fake.joblib')

In [2]:
agent = ExpeLAgent(
    llm=llm,
    self_reflect_llm=llm,
    action_llm=llm,
    experience_memory=ExpeLExperienceMemory(experiences=experiences)
)

In [3]:
experiences = agent.gather_experience(
    questions=hotpotqa.question.values[-5:],
    keys=hotpotqa.answer.values[-5:]
)

c:\Users\tuvin\anaconda3\envs\discussion-agents\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\tuvin\anaconda3\envs\discussion-agents\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [4]:
experiences

{'idxs': [0, 1, 2, 3, 4],
 'questions': ["VIVA Media AG changed it's name in 2004. What does their new acronym stand for?",
  'Which of Jonny Craig and Pete Doherty has been a member of more bands ?',
  'Where was the first governor after the The Missouri Compromise from?',
  'The creator of "Wallace and Gromit" also created what animation comedy that matched animated zoo animals with a soundtrack of people talking about their homes? ',
  "Woman's Era and Naj are what kind of magazines?"],
 'keys': ['Gesellschaft mit beschränkter Haftung',
  'Jonny" Craig',
  'Bath, Maine',
  'Creature Comforts',
  'fortnightly women interest magazine'],
 'trajectories': [[(False,
    'unable to find information',
    [('Thought: I need to search for VIVA Media AG and find out their new acronym after changing their name in 2004.',
      'Action: Search[VIVA Media AG]',
      "Observation 1: Could not find [VIVA Media AG]. Similar: ['MTV Music (Polish TV channel)', 'VIVA Plus', 'Paramount International 